---
title: "NautiloPy: An open-source Python framework for the Gironda Underwater
Cave Sonar And vision data set"
subtitle: Overview of the work realized with NautiloPy
authors:
  - name: Thomas Guilment
    affiliations:
      - University of Louisiana at Lafayette
    email: thomas.guilment@gmail.com
    corresponding: true
    orcid: 0009-0003-8163-3976
  - name: Gabriele Morra
    affiliations:
      - University of Louisiana at Lafayette
  - name: Leonardo Macelloni
    affiliations:
      - University of Mississippi
  - name: Marco D'Emidio
    affiliations:
      - University of Southern University
format:
  html:
    code-fold: true
---

![](./img/Logo_nautilopy_tiny_v2.png)

(theUnderwaterCaveDataset)=
## The underwater cave dataset
NautiloPy already includes a `csv.zip` file in the data folder, which is used for the entire project.  
The original data are available at [https://cirs.udg.edu/caves-dataset/](https://cirs.udg.edu/caves-dataset/). To access the corresponding article, see also [Maillos et al., 2017](doi:10.1177/0278364917732838).

### Abstract from the authors of the dataset

From [Maillos et al., 2017](doi:10.1177/0278364917732838)

"We provide a data set collected with an autonomous underwater vehicle (AUV) tested in the unstructured environment of an underwater cave complex. The vehicle is equipped with two mechanically scanned imaging sonar (MSIS) to simultaneously map the caves' horizontal and vertical surfaces, a Doppler velocity log (DvL), two inertial measurement units (IMUs), a depth sensor, and a vertically mounted camera imaging the sea-floor for ground truth validation in specific points. Here we present data using this testbed that was collected while guided by a diver, due to the caves' spatial complexity, during July 2013. For ease of use, the original ROS bag files are accompanied by a derivative version combining imagery and human-readable text files for processing on other environments."

:::{figure}
:label: my-figure
:align: left

(aerial-view)=
![The approximate path traveled underwater overlaid on an aerial image from Google Earth.](./img/CavesGoogleEarth2.jpg)

(auv-config)=
![AUV configuration](./img/auv_config.png)

Aerial image showing the AUV trajectory and AUV configuration from [Maillos et al., 2017](doi:10.1177/0278364917732838).
:::

::: {.callout-warning}
#### Correction in Super Seaking Reference axis
We believe that there is an error in the 3D axis representation of the vertical SONAR. The X-axis (in red) should point to the seafloor (not the sky). In this work, a 180° degrees rotation around the Z axis was applied.
```{image} ./img/auv_config_updated.jpg
:alt: Correction of the Seaking sonar
:width: 800px
:align: center
```
:::

:::{note} Dataset in details
:class: dropdown
%:header: Available data from sensors in CSV file
- **/depth_sensor:** DS2806 HPS-A pressure sensor data.
- **/dvl_linkquest:** LinkQuest NavQuest 600 sensor data. Contains bottom and water-referenced velocities and other important sensor data.
- **/imu_adis:** Analog Devices ADIS16480 sensor data. Contains orientation as both Euler angles and quaternion. It also contains raw data for accelerometers, gyros, magnetometers, and temperature, and an estimation of the gyro biases.
- **/imu_adis_ros:** Analog Devices ADIS16480 sensor orientation [using standard IMU ROS message.](http://wiki.ros.org/sensor_msgs)
- **/imu_xsens_mti:** Xsens MTi sensor data. Same message type and the same information as the topic /imu_adis.
- **/imu_xsens_mti_ros:** Xsens MTi sensor orientation using standard IMU ROS message.
- **/odometry:** estimation of the robot pose provided as a [standard Odometry ROS message.](http://wiki.ros.org/nav_msgs)
- **/sonar_micron:** Tritech Micron DST sensor beam data.
- **/sonar_micron_ros:** Micron data provided as [standard Laserscan ROS message.](http://wiki.ros.org/sensor_msgs)
- **/sonar_seaking:** Tritech Super SeaKing DFP profiler sensor beam data.
- **/sonar_seaking_ros:** profiler data provided as standard Laserscan ROS message.
- **/tf:** [standard ROS messages](http://wiki.ros.org/tf) containing sensor offsets.

```{figure} ./img/sensors_offset.png
:name: Sensors_offset
:align: left
Sensors offsets
```
:::

### Import Nautilopy

In [ ]:
from nautilopy.core import *

(prepareTheData)=
## Prepare the data
For completeness, instead of using the human format text file given by the dataset authors, the ROS bag data were extracted using the Python package `Bagpy` (see this [notebook](#ROS2csv) to see how it was done), and each topic was converted into CSV files. 

In this project, the odometry field is considered to be the best estimate of pose and orientation.
Therefore, the pre-processing is about interpolating the odometry to fit the vertical and horizontal sonar data (same timestamps).

Each processed data is saved as a pickle file.
For more information, don't hesitate to check the `01_Preprocessing.ipynb` notebook by clicking the [link below](#01_Preprocessing). 

:::{warning} Correction
:icon:False
During the data pre-processing phase, the vertical sonar orientation 3D axis has proven to be wrong, which is why there is a correction at the end of this external notebook.
:::

### Unzip the CSV dataset folder

In [ ]:
# If the data are not unzip
if not os.path.isdir("./data/csv/"):
    # Unizp the csv.zip file containing the CSV files
    f_unzip_file(s_zip_path = os.path.join("data", "csv.zip"), s_extract_path = os.path.join("data", '.'))
else:
    print("""The csv folder already exist.""")

### Create the Pickle dataset from the CSVs

In [ ]:
# Test if the pickle_dataset already exist
if not os.path.isdir("./data/Pickle_dataset/"):
    %run 01_Preprocessing.ipynb
    print("Done!")
else:
    print("""The folder Pickle_dataset already exists. 
If you want to recreate the Pickle dataset, delete the Pickle_dataset in the data folder, then re-run this cell.""")

### Load the pre-processed data
A little trick is used to get the `globals()` (local variables in the workspace) from an outside function by passing the local `globals` variables from the current notebook. The pickle variables are loaded in the current notebook. This is like loading a common Matlab `workspace`.

In [ ]:
# List the pickle files
l_var = os.listdir(os.path.join('.','data','Pickle_dataset'))

# Load only the needed variables
glob = globals()
f_load_var(os.path.join('.','data','Pickle_dataset'), glob, 
           ['m_xyz_pos', # XYZ AUV position
            'm_ypr', # Yaw, Pitch, Roll associated with the AUV positions
            'v_timestamp.pkl']) # Timestamps associated with AUV positions

# Deleting glob
del glob

(VisualizationOfTheAuvOrientationOverTheTrajectory)=
## Visualization of the AUV orientation over the trajectory
This part presents different solutions to visualize the AUV orientation over its trajectory.  
We are using the variable `m_xyz_pos` {eval}`m_xyz_pos.shape` and `m_ypr` {eval}`m_ypr.shape` created from the `odometry` data to display the trajectory and orientation of the AUV.

### Traditional multi-view

In [ ]:
# Inline mode that displays static images inside the notebook
%matplotlib inline
d_start = 9000      # Starting point (from 0 to np.shape(m_xyz_pos)[0])
d_end = 11000       # Ending point (maximum value is np.shape(m_xyz_pos)[0])
d_downsampling = 20 # For faster display, only shows 1 point every 20 points
d_step = 300        # (default 200) Show the pose every 300 divided by downsampled points
d_scale = 1.5       # (default 1.5) Length of the AUV 3D axis 
d_max_ticks = 8     # (default 5) number of ticks per axis

h = f_plot_auv_orientation_subplots(
    m_xyz_pos[d_start:d_end:d_downsampling], # XYZ AUV position
    m_ypr[d_start:d_end:d_downsampling], # Yaw, Pitch, Roll associated with the AUV positions
    d_step=int(d_step/d_downsampling), # updated step depending of downsampling
    d_scale = d_scale, 
    d_max_ticks = d_max_ticks
)

## Interactive AUV orientation over trajectory
This section presents two interactive plots of the Autonomous 3D orientation axis over its trajectory.
First, we use `Matplotlib`, then `plotly`.

In [ ]:
d_start = 0
d_end = np.shape(m_xyz_pos)[0]
d_downsampling = 10 # For faster and lighter display, only shows 1 point every 10 points
d_step = 400 # (default 200) Show the pose every 400 divided by downsampled points
d_scale = 3 # (default 1.5) Length of the AUV 3D axis 
d_max_ticks = 8 # (default 5) number of ticks per axis

v_figsize =(12, 5) # (default (12,5)) Width size and height size of the main figure

o_anim_auv_orientation = f_create_auv_orientation_animation(
    m_xyz_pos[d_start:d_end:d_downsampling],
    m_ypr[d_start:d_end:d_downsampling],
    d_step=int(d_step/d_downsampling),
    d_scale = d_scale,
    d_max_ticks = d_max_ticks,
    v_figsize = v_figsize
)

In [ ]:
# Convert the animation into javascript then HTML in one line code
display(HTML(o_anim_auv_orientation.to_jshtml()))

This code below allows to go back to `inline` backend (fixed plot) and clean the widget to prevent that the animation get too much memory.

In [ ]:
# Ensure to be back to the inline backend
%matplotlib inline

# Clean widgets in memory
plt.close('all')

Animation with plotly

In [ ]:
# Use the renderers "plotly_mimetype", "Jupyterlab", and "notebook" to ensure compatibility for the display animations
# available renderers at https://plot.ly/python/renderers/
import plotly.io as pio

# Use Jupyterlab compatible renderers
pio.renderers.default = "jupyterlab+notebook+plotly_mimetype"

In [ ]:
## User-defined parameters
d_start = 0
d_end = np.shape(m_xyz_pos)[0]
d_downsampling = 10
d_step = 200
d_max_ticks = 5

#d_step=int(d_step/d_downsampling)

# Apply slicing and downsampling
m_pos_sub = m_xyz_pos[d_start:d_end:d_downsampling].copy()
m_ypr_sub = m_ypr[d_start:d_end:d_downsampling].copy()
v_timestamp_sub = v_timestamp[d_start:d_end:d_downsampling].copy()

fig = f_create_auv_orientation_animation_plotly(m_pos_sub, m_ypr_sub, v_timestamp_sub, int(d_step/d_downsampling))
fig.show()

In [ ]:
# Ensure inline backend and no widgets are still "running"
plt.close('all')
%matplotlib inline

(VerticalAndHorizontalSonarDataVisualization)=
## Vertical and horizontal SONAR data visualization

This section presents different views of the sonar data:
- Static Polar view: received energy per angle
- Animated Polar view: received energy per range over time
- Sonar as an image of received energy per range over time
- Full view of sonar date with AUV trajectory path

<!-- The received signals can be seen over time as an image. Each waveform can be plotted as curves. The Y axis from top to bottom represents the distance from which the energy is received. The X axis is the time. -->

### Load the pre-processed sonar data

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob,
            ['m_beam_data_micron',       # (397, 45587) Received sonar data per range (397 bins from 0 to 20 meters) over time (associated with scanning angles) in dB
             'v_angles_rad_micron',      # associated scanning angle over time in radian
             'v_range_micron',           # Range scale from 0 to 20 over 397 bins
             'm_beam_data_micron_clean', # Cleaned sonar data
             'm_xyz_pos_micron',         # XYZ 3D position of the horizontal Micron sonar
             'v_timestamp_micron',
             'm_ypr_micron'])      # Timestamp associated with each sonar position
del glob

Polar and continous visualization of the horizontal and the vertical sonar

In [ ]:
# Starting when the sonar is 0º equivalent to the index 194 for the horizontal sonar
d_start_micron = 194

v_figsize = (8,8)

# Observing 1 period (200 points)
d_end_micron = d_start_micron + 200
m_sonar = m_beam_data_micron[:,d_start_micron:d_end_micron]
v_angle = v_angles_rad_micron[d_start_micron:d_end_micron]
f_plot_polar_sonar_micron(m_sonar, v_angle)

In [ ]:
d_image_width = 200
d_start_micron = 194
d_end_micron = d_start_micron + d_image_width
m_sonar_micron = m_beam_data_micron[:, d_start_micron:d_end_micron]
v_angles_rad = v_angles_rad_micron

v_figsize = (12,6)

f_plot_horizontal_sonar_with_angles(m_sonar_micron, v_angles_rad, d_start_micron, d_image_width, v_range_micron, v_figsize)

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob, 
            ['m_beam_data_seaking', 
             'v_angles_rad_seaking',
             'v_range_seaking',
             'm_xyz_pos_seaking',
             'v_timestamp_seaking',
             'm_beam_data_seaking_clean'])
del glob

In [ ]:
# Starting when the sonar is 0º equivalent to the index 95 for the vertical sonar
d_start_seaking = 95

# Observing 1 period (201 points)
d_end_seaking = d_start_seaking + 201

# Figure size
v_figsize = (8,8)

# Extracting the wanted part
m_sonar = m_beam_data_seaking[:,d_start_seaking:d_end_seaking]
v_angle = v_angles_rad_seaking[d_start_seaking:d_end_seaking]
f_plot_polar_sonar_seaking(m_sonar, v_angle, v_figsize)

In [ ]:
d_image_width = 202 # 1 period of 201 index + 1 point to display the full 360º

# Starting when the sonar is 0º equivalent to the index 95 for the vertical sonar
d_start_seaking = 95 
d_end_seaking = d_start_seaking + d_image_width

m_sonar_seaking = m_beam_data_seaking[:, d_start_seaking:d_end_seaking]
v_angles_rad = v_angles_rad_seaking
f_plot_vertical_sonar_with_angles(m_sonar_seaking, v_angles_rad, d_start_seaking, d_image_width, v_range_seaking)

## Observing several sonar revolutions

At the beginning of the survey...

In [ ]:
d_image_width = 1000
d_start_micron = 194
# d_end_micron = d_start_micron + d_image_width
m_sonar_micron = m_beam_data_micron[:, d_start_micron:d_start_micron+d_image_width]
v_figsize = (16,6)
f_plot_horizontal_sonar(m_sonar_micron, d_start_micron, d_image_width, v_range_micron, v_figsize)

In [ ]:
d_image_width = 800
d_start_micron = 194 # Angle is 0 rad at index 194 and period every 200

d_end_micron = d_start_micron + d_image_width

# Put the seaking start accordingly to the micron for consistency
d_start_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_start_micron]))
d_end_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_end_micron]))

d_image_width = d_end_seaking - d_start_seaking + 1

# d_start_seaking = 95
# d_end_seaking = d_start_seaking + d_image_width
m_sonar_seaking = m_beam_data_seaking[:, d_start_seaking:d_start_seaking+d_image_width]
v_figsize = (16,6)
f_plot_vertical_sonar(m_sonar_seaking, d_start_seaking, d_image_width, v_range_seaking, v_figsize)

In the middle of the survey...

In [ ]:
d_image_width = 800
d_start_micron = 194 + 22400  # Angle is 0 rad at index 194 and period every 200
# d_end_micron = d_start_micron + d_image_width
m_sonar_micron = m_beam_data_micron[:, d_start_micron:d_start_micron+d_image_width]
v_figsize = (16,6)
f_plot_horizontal_sonar(m_sonar_micron, d_start_micron, d_image_width, v_range_micron, v_figsize)

In [ ]:
d_start_micron = 3800+194  # Angle is 0 rad at index 194 and period every 200
d_end_micron = d_start_micron + 1400
m_sonar_micron = m_beam_data_micron[:, d_start_micron:d_end_micron]
f_plot_horizontal_sonar(m_sonar_micron, d_start_micron, d_image_width, v_range_micron)

In [ ]:
d_image_width = 800
d_start_micron = 194 + 22400 # Angle is 0 rad at index 194 and period every 200

d_end_micron = d_start_micron + d_image_width

# Put the seaking start accordingly to the micron for consistency
d_start_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_start_micron]))
d_end_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_end_micron]))

d_image_width = d_end_seaking - d_start_seaking + 1

m_sonar_seaking = m_beam_data_seaking[:, d_start_seaking:d_start_seaking+d_image_width]
v_figsize = (16,6)
f_plot_vertical_sonar(m_sonar_seaking, d_start_seaking, d_image_width, v_range_seaking, v_figsize)

### Interactive analysis
Widget to update the part of the trajectory in color associated with the sonar image below

In [ ]:
plt.close('all')
%matplotlib inline

In [ ]:
f_micron_with_trajectory(m_xyz_pos_micron, m_beam_data_micron, v_range_micron)

In [ ]:
f_seaking_with_trajectory(m_xyz_pos_seaking, m_beam_data_seaking, v_range_seaking)

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob, 
            ['m_beam_data_seaking', 
             'v_angles_rad_seaking',
             'v_range_seaking',
             'm_xyz_pos_seaking',
             'm_ypr_seaking',
             'v_timestamp_seaking',
             'm_beam_data_seaking_clean'])
del glob

## Animation scanning sonar 

In [ ]:
d_start_seaking = 32456
d_end_seaking = d_start_seaking + 2000

fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(projection="3d", proj_type="ortho")

# Put the axis in color and names them (X,Y,Z)
colors = ("#FF6666", "#005533", "#1199EE")  # Colorblind-safe RGB

for i, (axis, c) in enumerate(zip((ax.xaxis, ax.yaxis, ax.zaxis),
                                  colors)):
    axlabel = axis.axis_name
    axis.set_label_text(axlabel)
    axis.label.set_color(c)
    axis.line.set_color(c)
    axis.set_tick_params(colors=c)

# Plot trajectory with beginning and end
m_pos_seaking = m_xyz_pos_seaking[d_start_seaking:d_end_seaking,:].copy()
m_pos_seaking[:,2] = -m_pos_seaking[:,2]

trajectory = m_pos_seaking.copy()

# Setting the axes properties
d_margin = 15
ax.set_xlim3d([np.min(trajectory[:,0])-d_margin, np.max(trajectory[:,0])+d_margin])
ax.set_xlabel('X')

ax.set_ylim3d([np.min(trajectory[:,1])-d_margin, np.max(trajectory[:,1])+d_margin])
ax.set_ylabel('Y')

ax.set_zlim3d([np.min(trajectory[:,2])-d_margin, np.max(trajectory[:,2])+d_margin])
ax.set_zlabel('Z')

f_plot_3d_trajectory(ax,m_pos_seaking)
ax.set_title("(y,z) view vertical sonar scanning (Starting at 0°)")

m_aux_pc_seaking,l_intensity_seaking = f_pointCloud(
        m_xyz_pos_seaking[d_start_seaking:d_end_seaking,:], 
        m_ypr_seaking[d_start_seaking:d_end_seaking,:], 
        m_beam_data_seaking_clean[:,d_start_seaking:d_end_seaking], 
        v_angles_rad_seaking[d_start_seaking:d_end_seaking], 
        d_max_range=10, 
        d_thresh=0,b_return_intensity=True)

seaking_color = np.concatenate(l_intensity_seaking)

global p_pos
global graph
d_ind=0
p_pos = ax.scatter(m_pos_seaking[d_ind,0],m_pos_seaking[d_ind,1],m_pos_seaking[d_ind,2],color='blue')

# ax.legend()
ax.view_init(20,65)

plt.close(fig)

graph = ax.scatter(m_aux_pc_seaking[:,0], m_aux_pc_seaking[:,1], m_aux_pc_seaking[:,2], s=1, c=seaking_color, cmap='jet')


l_index_seaking = [len(v_elem) for v_elem in l_intensity_seaking]
l_index_seaking = np.concatenate(([0],np.cumsum(l_index_seaking)))

def f_update_anim_sonar(d_ind):
    global graph
    global p_pos
    graph.remove()
    p_pos.remove()
    # Create the plot and get the graph object
    graph = ax.scatter(m_aux_pc_seaking[:l_index_seaking[d_ind],0], m_aux_pc_seaking[:l_index_seaking[d_ind],1], m_aux_pc_seaking[:l_index_seaking[d_ind],2], s=1, c=seaking_color[:l_index_seaking[d_ind]], cmap='jet')
    p_pos = ax.scatter(m_pos_seaking[d_ind,0],m_pos_seaking[d_ind,1],m_pos_seaking[d_ind,2],color='yellow')
    fig.canvas.flush_events()
    fig.canvas.draw_idle()
    fig.canvas.draw()
    display(fig)

In [ ]:
widget_seaking_3D = interactive(
    f_update_anim_sonar,
    d_ind=widgets.IntSlider(min=1,max=d_end_seaking-d_start_seaking-1,step=1,value=10)) # widgets.FloatSlider(min=0,max=1,step=0.05,value=0.9))

In [ ]:
display(widget_seaking_3D)

In [ ]:
plt.close('all')
%matplotlib inline

## Cleaning the SONAR data

#### Mask creation from intensity
When the wall detection is close by, removing the body frame detection with a simple rectangle could also remove the detected wall.
The idea is to create a mask of the body frame detection with intensity and subtract this intensity from the total data.
In the case of font body detection, the wall detection doesn't seem to overlap the AUV body frame detection so a simple replacement of values should suffice.  

After manually checking the data, we selected the index 594 and 4064 for the rigging echoes of the back and front AUV body respectively.
This method can be improved, this is simply to show one way of processing the sonar noise.

In [ ]:
m_mask_instensity_back = m_beam_data_micron[0:70, 594 - 35 : 594 + 35].copy()
d_length_back_beam = 70
d_length_back_start = -35
d_length_back_end = 35

m_mask_intensity_front = m_beam_data_micron[0:30, 4064:4134].copy()
d_length_front_start = -35
d_length_front_end = 35
d_length_front_beam = 30

In [ ]:
center_back = 400 + 194 # 594
center_front = 3770 + 194 + 100 # 4064
window_half_width = 35

d_length_back_beam = 70
d_length_front_beam = 30

v_range_back = v_range_micron[:d_length_back_beam]
v_range_front = v_range_micron[:d_length_front_beam]

# Plot: Horizontal Sonar rigging echoes
plt.style.use('default')
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Back echo image
ax[0].set_title("Horizontal Sonar rigging echoes on the back AUV body", fontsize=16)

extent_back = [
    0, 2*window_half_width,
    v_range_back[-1], v_range_back[0]   # Flip so 0 m is at top
]

h_data_back = m_beam_data_micron[0:d_length_back_beam,
                                 center_back - window_half_width : center_back + window_half_width]

h_im = ax[0].imshow(
    h_data_back,
    aspect='auto',
    extent=extent_back,
    interpolation='none'
)

ax[0].set_xlabel("Received Beam Index Offset", fontsize=14)
ax[0].set_ylabel("Range (meters)", fontsize=14)
ax[0].tick_params(labelsize=14)

cbar = plt.colorbar(h_im, ax=ax[0])
cbar.set_label("Intensity (in dB)", fontsize=14)
cbar.ax.tick_params(labelsize=14)

# Front echo image
ax[1].set_title("Horizontal Sonar rigging echoes on the front AUV body", fontsize=16)

extent_front = [
    0, 2*window_half_width,
    v_range_front[-1], v_range_front[0]
]

h_data_front = m_beam_data_micron[0:d_length_front_beam,
                                  center_front - window_half_width : center_front + window_half_width]

h_im2 = ax[1].imshow(
    h_data_front,
    aspect='auto',
    extent=extent_front,
    interpolation='none'
)

ax[1].set_xlabel("Received Beam Index Offset", fontsize=14)
ax[1].set_ylabel("Range (meters)", fontsize=14)
ax[1].tick_params(labelsize=14)

cbar2 = plt.colorbar(h_im2, ax=ax[1])
cbar2.set_label("Intensity (in dB)", fontsize=14)
cbar2.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 7))
ax[0].set_title("Horizontal Sonar back AUV body frame rigging echoes")
h_im_mask_back_auv = ax[0].imshow(m_beam_data_micron[0:d_length_back_beam, 594 + d_length_back_start : 594 + d_length_back_end], 
             extent = [0, 2*d_length_back_end, 
            v_range_micron[d_length_back_beam], v_range_micron[0]], aspect='auto')   # Flip so 0 m is at top ])
ax[0].set_ylabel("Range (meters)", fontsize=14)
ax[0].tick_params(labelsize=14)

ax[1].set_title("Horizontal Sonar front AUV body frame rigging echoes")
h_im_mask_front_auv = ax[1].imshow(m_beam_data_micron[0:d_length_front_beam, 4099 + d_length_front_start : 4099 + d_length_front_end],
             extent = [0, 2*d_length_front_end, 
            v_range_micron[d_length_front_beam], v_range_micron[0]], aspect='auto');   # Flip so 0 m is at top ])
ax[1].set_ylabel("Range (meters)", fontsize=14)
ax[1].tick_params(labelsize=14)

plt.tight_layout()
plt.show()

The cleaning process is already done in the Pre-processing notebook. Below are the results. Don't hesitate to go check the preprocessing notebook clicking on this preview llink: [](#01_Preprocessing)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.close('all')

d_start_micron = 22994
d_end_micron = d_start_micron + 1000 + 1  # +1 to include the 1000th column

# Extract range axis
v_range = v_range_micron  # assumed to be 1D of shape (num_rows,)
v_y_extent = [v_range[-1], v_range[0]]  # flip so 0 m is at top

# X-axis extent (optional, to show ping index if desired)
v_x_extent = [d_start_micron, d_end_micron]

fig, ax = plt.subplots(2, 1, figsize=(12, 7))

# Before cleaning
ax[0].set_title("Before cleaning", fontsize=16)
im0 = ax[0].imshow(
    m_beam_data_micron[:, d_start_micron:d_end_micron],
    aspect='auto',
    extent=[v_x_extent[0], v_x_extent[1], v_y_extent[0], v_y_extent[1]],
    interpolation='none'
)
# ax[0].set_xlabel("Received beam Index", fontsize=14)
ax[0].set_ylabel("Range (meters)", fontsize=14)
ax[0].tick_params(labelsize=12)
plt.colorbar(im0, ax=ax[0], label="Intensity (dB)")

# After cleaning
ax[1].set_title("After cleaning", fontsize=16)
im1 = ax[1].imshow(
    m_beam_data_micron_clean[:, d_start_micron:d_end_micron],
    aspect='auto',
    extent=[v_x_extent[0], v_x_extent[1], v_y_extent[0], v_y_extent[1]],
    interpolation='none'
)
ax[1].set_xlabel("Received beam Index", fontsize=14)
ax[1].set_ylabel("Range (meters)", fontsize=14)
ax[1].tick_params(labelsize=12)
plt.colorbar(im1, ax=ax[1], label="Intensity (dB)")

plt.tight_layout()
plt.show()

### Hysteresis thresholding

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob, 
            ['m_beam_data_seaking_clean_hyst', 
             'm_beam_data_micron_clean_hyst'])
del glob

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.close('all')

d_start_micron = 22994
d_end_micron = d_start_micron + 1000 + 1  # +1 to include the 1000th column

# Extract range axis
v_range = v_range_micron  # assumed to be 1D of shape (num_rows,)
v_y_extent = [v_range[-1], v_range[0]]  # flip so 0 m is at top

# X-axis extent (optional, to show ping index if desired)
v_x_extent = [d_start_micron, d_end_micron]

fig, ax = plt.subplots(2, 1, figsize=(12, 7))

# Before cleaning
ax[0].set_title("Cleaned data", fontsize=16)
im0 = ax[0].imshow(
    m_beam_data_micron_clean[:, d_start_micron:d_end_micron],
    aspect='auto',
    extent=[v_x_extent[0], v_x_extent[1], v_y_extent[0], v_y_extent[1]],
    interpolation='none'
)
# ax[0].set_xlabel("Received beam Index", fontsize=14)
ax[0].set_ylabel("Range (meters)", fontsize=14)
ax[0].tick_params(labelsize=12)
plt.colorbar(im0, ax=ax[0], label="Intensity (dB)")

# After cleaning
ax[1].set_title("Hysteresis thresholding", fontsize=16)
im1 = ax[1].imshow(
    m_beam_data_micron_clean_hyst[:, d_start_micron:d_end_micron],
    aspect='auto',
    extent=[v_x_extent[0], v_x_extent[1], v_y_extent[0], v_y_extent[1]],
    interpolation='none'
)
ax[1].set_xlabel("Received beam Index", fontsize=14)
ax[1].set_ylabel("Range (meters)", fontsize=14)
ax[1].tick_params(labelsize=12)
plt.colorbar(im1, ax=ax[1], label="Intensity (dB)")

plt.tight_layout()
plt.show()

## Hysteresis animation

In [ ]:
d_start_micron = 22994
d_end_micron = d_start_micron + 1000

m_image = (m_beam_data_micron_clean[:, d_start_micron:d_end_micron]- np.min(m_beam_data_micron_clean[:, d_start_micron:d_end_micron])) / (np.max(m_beam_data_micron_clean[:, d_start_micron:d_end_micron])- np.min(m_beam_data_micron_clean[:, d_start_micron:d_end_micron]))

slider_low = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.4)
slider_high = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.7)
slider_overlap = widgets.FloatSlider(min=0.01, max=1, step=0.01, value=0.5)
slider_window_size = widgets.IntSlider(min=2, max=1000, step=1, value=200)

h_fig_hyst = plt.figure(figsize=(10, 7))
ax_orig = h_fig_hyst.add_subplot(211)
ax_new = h_fig_hyst.add_subplot(212)

v_range_crop = v_range_micron[0:m_image.shape[0]]

extent = [
    22994,                       # x-axis: column index start
    22994+m_image.shape[1],        # x-axis: column index end
    v_range_crop[-1],        # y-axis: bottom (flipped so 0 m is at top)
    v_range_crop[0],         # y-axis: top
]

h_orig = ax_orig.imshow(m_image, aspect='auto', extent=extent, interpolation='none')

# h_orig = ax_orig.imshow(m_image, aspect='auto', extend)  # , cmap="gray",vmin=0,vmax=1)
ax_orig.set_title("Cleaned Sonar data", fontsize=14)
#ax_orig.set_xlabel("Beam Index", fontsize=16)
ax_orig.set_ylabel("Range (meters)", fontsize=14)

h_new = ax_new.imshow(m_image, aspect='auto', extent=extent, interpolation='none')  # , cmap="gray",vmin=0,vmax=1)
ax_new.set_title("Hysteresis threshold", fontsize=14)
ax_new.set_xlabel("Beam Index", fontsize=14)
ax_new.set_ylabel("Range (meters)", fontsize=12)
plt.close(h_fig_hyst)

def f_hyster_update(m_image, d_low, d_high, d_overlap=0.5, d_window_size=100):
    m_aux_tot = m_image.copy()

    for d_ind in range(
        0, np.shape(m_image)[1], int(np.round(d_window_size * d_overlap))
    ):
        m_aux_loc = m_image[:, d_ind : d_ind + d_window_size].copy()

        # Low and high threshold according to a percentage of the local maximum value
        # d_high_thresh = np.max(m_aux_sonar_micron)*0.75
        # d_low_thresh = np.max(m_aux_sonar_micron)*0.25
        # Low and high threshold according to a percentage of the local maximum value

        d_high_thresh = (np.max(m_aux_loc) * d_high)  # Remove very low intensity localy compare to global intensity
        d_low_thresh = np.max(m_aux_loc) * d_low

        m_hyst = filters.apply_hysteresis_threshold(m_aux_loc, d_low_thresh, d_high_thresh)

        m_aux = np.zeros(np.shape(m_aux_loc))
        m_aux[m_hyst] = m_aux_loc[m_hyst]

        m_aux_tot[:, d_ind : d_ind + int(np.round(d_window_size * d_overlap))] = m_aux[:, 0 : int(np.round(d_window_size * d_overlap))]

    # m_hyst = filters.apply_hysteresis_threshold(m_image, d_low, d_high)

    # m_aux = np.zeros(np.shape(m_image))
    # m_aux[m_hyst] = m_image[m_hyst].copy()

    h_new.set_data(m_aux_tot)
    # ax_new.imshow(m_aux, cmap="gray")
    h_fig_hyst.canvas.flush_events()
    h_fig_hyst.canvas.draw_idle()
    # fig.canvas.draw()
    # fig.draw()
    display(h_fig_hyst)
    # return m_aux_tot

In [ ]:
widget_micron_hyst = interactive(
    f_hyster_update,
    m_image = widgets.fixed(m_image),
    d_low = slider_low,
    d_high = slider_high,
    d_overlap = slider_overlap,
    d_window_size = slider_window_size,
)

In [ ]:
display(widget_micron_hyst)

## 3D point cloud from combining both SONAR data with a uniform colormap based on cleaned data

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob, ['v_timestamp_seaking', 'v_timestamp_micron','m_ypr_micron','m_ypr_seaking'])
del glob

In [ ]:
d_start_micron = 0 
d_end_micron =  d_start_micron + np.shape(m_beam_data_micron_clean)[1]-1 #6500

# Put the seaking start accordingly to the micron one
d_start_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_start_micron]))
d_end_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_end_micron]))

# The max values are selected when d_tresh = -1
m_point_cloud_micron_max = f_pointCloud(
    m_xyz_pos_micron[d_start_micron:d_end_micron,:], 
    m_ypr_micron[d_start_micron:d_end_micron,:], 
    m_beam_data_micron_clean[:,d_start_micron:d_end_micron], 
    v_angles_rad_micron[d_start_micron:d_end_micron], 
    d_max_range=20, 
    d_thresh=-1)

m_point_cloud_seaking_max = f_pointCloud(
    m_xyz_pos_seaking[d_start_seaking:d_end_seaking,:], 
    m_ypr_seaking[d_start_seaking:d_end_seaking,:], 
    m_beam_data_seaking_clean[:,d_start_seaking:d_end_seaking], 
    v_angles_rad_seaking[d_start_seaking:d_end_seaking], 
    d_max_range=10, 
    d_thresh=-1)

In [ ]:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(25,12))
ax = fig.add_subplot(projection="3d", proj_type="ortho",box_aspect=(5,4,2))

# Put the axis in color and name them (X,Y,Z)
colors = ("#FF6666", "#005533", "#1199EE")  # Colorblind-safe RGB
for i, (axis, c) in enumerate(zip((ax.xaxis, ax.yaxis, ax.zaxis),
                                  colors)):
    axlabel = axis.axis_name
    axis.set_label_text(axlabel)
    axis.label.set_color(c)
    axis.line.set_color(c)
    axis.set_tick_params(colors=c)

m_pos_micron = m_xyz_pos_micron[d_start_micron:d_end_micron,:].copy()
m_pos_micron[:,2] = -m_pos_micron[:,2]

m_pos_seaking = m_xyz_pos_seaking[d_start_seaking:d_end_seaking,:].copy()
m_pos_seaking[:,2] = -m_pos_seaking[:,2]

micron_color = m_point_cloud_micron_max[:,2 ] #np.concatenate(l_intensity_micron)
seaking_color = m_point_cloud_seaking_max[:,2]

# Put the same max and min value to have the same color map
d_min = np.min((np.min(micron_color),np.min(seaking_color)))
d_max = np.max((np.max(micron_color),np.max(seaking_color)))
micron_color[micron_color == np.min(micron_color)] = d_min
seaking_color[seaking_color == np.min(seaking_color)] = d_min

micron_color[micron_color == np.max(micron_color)] = d_max
seaking_color[seaking_color == np.max(seaking_color)] = d_max

# Plot trajectory with beginning and end
# f_plot_3d_trajectory(ax,m_pos_micron)
f_plot_3d_trajectory(ax,m_pos_seaking)

p = ax.scatter(m_point_cloud_micron_max[:,0], m_point_cloud_micron_max[:,1], m_point_cloud_micron_max[:,2], s=1, c=micron_color, cmap='jet', label='Micron')
p = ax.scatter(m_point_cloud_seaking_max[:,0], m_point_cloud_seaking_max[:,1], m_point_cloud_seaking_max[:,2], s=1, c=seaking_color, cmap='jet', label='Seaking')

ax.set_title("Perspective view map ")
# ax.legend()
ax.view_init(30,45)
ax.invert_yaxis()

ax.set_title("3D Point cloud from Horizontal and vertical SONAR", fontsize=30)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.dist = 0
fig.colorbar(p,shrink=0.6)
plt.show()

## Animated 3D mapping using plotly

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Data exactly as before
m_pos_micron = m_xyz_pos_micron[d_start_micron:d_end_micron, :].copy()
m_pos_micron[:, 2] = -m_pos_micron[:, 2]          # invert Z

m_pos_seaking = m_xyz_pos_seaking[d_start_seaking:d_end_seaking, :].copy()
m_pos_seaking[:, 2] = -m_pos_seaking[:, 2]        # invert Z

micron_color  = m_point_cloud_micron_max[:, 2].copy()
seaking_color = m_point_cloud_seaking_max[:, 2].copy()

# match the colour scale across both clouds
d_min = np.min((micron_color.min(), seaking_color.min()))
d_max = np.max((micron_color.max(), seaking_color.max()))

micron_color[micron_color == micron_color.min()] = d_min
seaking_color[seaking_color == seaking_color.min()] = d_min
micron_color[micron_color == micron_color.max()] = d_max
seaking_color[seaking_color == seaking_color.max()] = d_max

# Build Plotly 3-D traces
micron_cloud = go.Scatter3d(
    x=m_point_cloud_micron_max[:, 0],
    y=m_point_cloud_micron_max[:, 1],
    z=m_point_cloud_micron_max[:, 2],
    mode="markers",
    name="Micron",
    marker=dict(
        size=2,
        color=micron_color,
        colorscale="Jet",
        cmin=d_min,
        cmax=d_max,
        showscale=False,          # single shared colorbar → put it on Seaking only
    )
)

seaking_cloud = go.Scatter3d(
    x=m_point_cloud_seaking_max[:, 0],
    y=m_point_cloud_seaking_max[:, 1],
    z=m_point_cloud_seaking_max[:, 2],
    mode="markers",
    name="Seaking",
    marker=dict(
        size=2,
        color=seaking_color,
        colorscale="Jet",
        cmin=d_min,
        cmax=d_max,
        colorbar=dict(title="Intensity", len=0.6),
    )
)

# Trajectory into a line trace
trajectory = go.Scatter3d(
    x=m_xyz_pos[:, 0],
    y=m_xyz_pos[:, 1],
    z=-m_xyz_pos[:, 2],
    mode="lines",
    name="AUV trajectory",
    line=dict(width=4, color="black"),
)

# Stacking all points into one array
all_points = np.vstack([
    m_point_cloud_micron_max,
    m_point_cloud_seaking_max,
    m_pos_seaking  # optional: include trajectory
])

# Compute center of the bounding box
center = all_points.mean(axis=0)
x_center, y_center, z_center = center

# Compute max range (to help calculate how far back to position camera)
ranges = all_points.max(axis=0) - all_points.min(axis=0)
max_range = np.max(ranges)

camera = dict(
    center=dict(x=0, y=0, z=0),  # Center stays on (0,0,0) in Plotly logic
    eye=dict(
        x=1.2 * max_range / 2,
        y=1.2 * max_range / 2,
        z=1.0 * max_range / 2,
    ),
    projection=dict(type="orthographic")
)

# Layout: axis colours, aspect ratio, orthographic camera, etc.
layout = go.Layout(
    title="3D Point cloud from Horizontal and Vertical SONAR",
    width=1200,   # <-- increase width
    height=800,   # <-- increase height
    legend=dict(itemsizing="constant"),
    scene=dict(
        aspectmode="manual",
        aspectratio=dict(x=2, y=1.6, z=0.8),
        camera=camera,
        # camera=dict(
        #     eye=dict(x=1.3, y=1.3, z=1.0),          # ≈ view_init(30,45)
        #     projection=dict(type="orthographic")     # ‘ortho’ projection
        # ),
        xaxis=dict(
            title="X",
            titlefont=dict(color="#FF6666"),
            tickfont=dict(color="#FF6666"),
            tickcolor="#FF6666",
        ),
        yaxis=dict(
            title="Y",
            titlefont=dict(color="#005533"),
            tickfont=dict(color="#005533"),
            tickcolor="#005533",
            autorange="reversed",                    # invert_yaxis()
        ),
        zaxis=dict(
            title="Z",
            titlefont=dict(color="#1199EE"),
            tickfont=dict(color="#1199EE"),
            tickcolor="#1199EE",
        ),
    ),
)

# Compose the figure and display
fig = go.Figure(data=[micron_cloud, seaking_cloud, trajectory], layout=layout)
fig.show()

## Miscellaneous
### Showing a function documentation
NautiloPy have very detailed function documentation writen in Mystmd that can be displayed using `f_show_doc()`.
Example on the quaternion to roll, pitch and Yaw `f_q2rollPitchYaw` function.

In [ ]:
f_show_doc(f_q2rollPitchYaw)

::::{note} Link to external notebooks
(01_Preprocessing)=
:::{card} 01_Preprocessing.ipynb
:link: ./01_Preprocessing.ipynb
See all the processes that have been done on the data from the CSV dataset to the pickle file
:::

(ROS2csv)=
:::{card} ROSbag2csv.ipynb
:link: ./ROSbag2csv.ipynb
See how to extract ROS bag data and convert each topic to a CSV file.
:::
::::